In [ ]:
import os
from typing import Optional, Dict, Any
import pandas as pd
from pymongo import MongoClient

# read_mongodb.py

MONGO_URI = os.getenv("MONGODB_URI", "mongodb://localhost:27017")
MONGO_DB = os.getenv("MONGODB_DB", "myappdb")
MONGO_COLLECTION = os.getenv("MONGODB_COLLECTION", "registrations")

def fetch_collection(
    uri: str = MONGO_URI,
    db_name: str = MONGO_DB,
    collection_name: str = MONGO_COLLECTION,
    query: Optional[Dict[str, Any]] = None,
    projection: Optional[Dict[str, int]] = None,
    limit: Optional[int] = None,
) -> pd.DataFrame:
    """
    Fetch documents from a MongoDB collection and return as a pandas DataFrame.
    """
    client = MongoClient(uri)
    try:
        coll = client[db_name][collection_name]
        cursor = coll.find(query or {}, projection)
        if limit:
            cursor = cursor.limit(limit)
        docs = list(cursor)
    finally:
        client.close()

    # Convert ObjectId to str for JSON/DF friendliness
    for d in docs:
        if "_id" in d:
            d["_id"] = str(d["_id"])

    return pd.DataFrame(docs)

if __name__ == "__main__":
    df = fetch_collection(limit=100)
    print(f"Loaded {len(df)} documents")
    display(df.head())

Loaded 100 documents


,_id,reg_id,event_id,attendee_email,reg_timestamp,payment_status,ticket_type
0,6978069ff37d946d8dc6c357,RE001000,E001,user0@example.com,2025-12-31 10:00,paid,earlybird
1,6978069ff37d946d8dc6c358,RE001001,E001,user1@example.com,2025-12-18 10:00,cancelled,standard
2,6978069ff37d946d8dc6c359,RE001002,E001,user2@example.com,2025-12-23 10:00,pending,student
3,6978069ff37d946d8dc6c35a,RE001003,E001,user3@example.com,2026-01-11 10:00,paid,student
4,6978069ff37d946d8dc6c35b,RE001004,E001,user4@example.com,2025-12-14 10:00,pending,standard
